In [ ]:
!pip install watermark

In [ ]:
%load_ext watermark

In [ ]:
%watermark -a "Rishit Kapoor 21BAI1046" -u -d -v -m

Author: Rishit Kapoor 21BAI1046

Last updated: 2024-02-23

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.58+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'traffic-sign-dataset-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1794080%2F2952603%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240223%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240223T071759Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D97644cf96407d50e09ca84cee41599da049cee6b48fd2ef7f9ca8a5f7aa64b325f1f2d09c55fcb64bfd097bd1fd6259a1dddd7a2f5ed710f1d3b9b12efedae5953e9942e10a8029f3c14dc89ba2800c087b40b5aac61dc74bb933ccccff6c99744a19814ecd034729a3edd3f4aeef0d772619252578a2d0ef9c0bee1bcee3dc3e0d22195b4b64377e7677ba9e454d0ab149ed8d8bd637b63d352cecec6d81ebb8cd20fdba1bd86159b6e1b0e06535e9daabc0cc01804df5694e4974e53ba03d64dc40a11ef36194bcc8c0652c2727b4e964bbbd64620e4ddfbb8288d844c44cacea34c9ef7a89898cc2dafe91e2fdb12f8d6fe3e39e46870ffdd7c988cc06e34'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 199059936 bytes downloaded
Downloaded and uncompressed: traffic-sign-dataset-classification
Data source import complete.


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.applications.resnet50 import ResNet50
from skimage.io import  imread, imshow
from skimage.transform import  resize, rescale
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
traindata_gen = ImageDataGenerator(zoom_range=0.5, shear_range=0.8, horizontal_flip=True, rescale=1/255)

In [ ]:
path='/kaggle/input/traffic-sign-dataset-classification/traffic_Data/DATA'
data_train_gen =traindata_gen.flow_from_directory(
    directory=path,
    target_size=(224,224),
    batch_size=3,
    class_mode="categorical",
    color_mode='rgb',
    seed = 1234,
    shuffle = True
    )

Found 4170 images belonging to 58 classes.


In [ ]:
data_train_gen.class_indices

{'0': 0,
 '1': 1,
 '10': 2,
 '11': 3,
 '12': 4,
 '13': 5,
 '14': 6,
 '15': 7,
 '16': 8,
 '17': 9,
 '18': 10,
 '19': 11,
 '2': 12,
 '20': 13,
 '21': 14,
 '22': 15,
 '23': 16,
 '24': 17,
 '25': 18,
 '26': 19,
 '27': 20,
 '28': 21,
 '29': 22,
 '3': 23,
 '30': 24,
 '31': 25,
 '32': 26,
 '33': 27,
 '34': 28,
 '35': 29,
 '36': 30,
 '37': 31,
 '38': 32,
 '39': 33,
 '4': 34,
 '40': 35,
 '41': 36,
 '42': 37,
 '43': 38,
 '44': 39,
 '45': 40,
 '46': 41,
 '47': 42,
 '48': 43,
 '49': 44,
 '5': 45,
 '50': 46,
 '51': 47,
 '52': 48,
 '53': 49,
 '54': 50,
 '55': 51,
 '56': 52,
 '57': 53,
 '6': 54,
 '7': 55,
 '8': 56,
 '9': 57}

In [ ]:
all_images = []
for img_path in glob.glob("/kaggle/input/traffic-sign-dataset-classification/traffic_Data/TEST/*"):
    img = Image.open(img_path)
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array = img_array / 255.0  # divide by 255.0 to get float values between 0 and 1 (Rescale)
    all_images.append(img_array)

# Resnet Pooling and filters #

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Conv2D, MaxPooling2D

In [ ]:
ResNet50_model = ResNet50(weights='imagenet',
                  include_top=False,
                  input_shape=(224,224,3))

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
x = Conv2D(32, (3, 3), activation='relu')(ResNet50_model.output)

# Add a MaxPooling layer
x = MaxPooling2D(pool_size=(2, 2))(x)

# Add Global Average Pooling layer
x = GlobalAveragePooling2D()(x)

# Flatten layer
f1 = Flatten()(x)

# Dense layer with Softmax activation
final_layer = Dense(58, activation='softmax')(f1)

In [ ]:
ResNet50_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [ ]:
model=Model(inputs=ResNet50_model.input,outputs=final_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_training = model.fit_generator(data_train_gen, epochs=15)

Epoch 1/15
1390/1390 [==============================] - 115s 63ms/step - loss: 3.1909 - accuracy: 0.2168
Epoch 2/15
1390/1390 [==============================] - 90s 65ms/step - loss: 2.3488 - accuracy: 0.3662
Epoch 3/15
1390/1390 [==============================] - 89s 64ms/step - loss: 1.9649 - accuracy: 0.4307
Epoch 4/15
1390/1390 [==============================] - 89s 64ms/step - loss: 1.7144 - accuracy: 0.5086
Epoch 5/15
1390/1390 [==============================] - 89s 64ms/step - loss: 1.4794 - accuracy: 0.5811
Epoch 6/15
1390/1390 [==============================] - 89s 64ms/step - loss: 1.2709 - accuracy: 0.6564
Epoch 7/15
1390/1390 [==============================] - 88s 63ms/step - loss: 1.1313 - accuracy: 0.6839
Epoch 8/15
1390/1390 [==============================] - 90s 65ms/step - loss: 0.9546 - accuracy: 0.7376
Epoch 9/15
1390/1390 [==============================] - 89s 64ms/step - loss: 0.8451 - accuracy: 0.7681
Epoch 10/15
1390/1390 [==============================] - 89s 64